In [48]:
import keras as K
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers.merge import Concatenate
import numpy as np
import sys
sys.path.append("./utils/")
from data_path import *
from os.path import join
from sklearn.model_selection import train_test_split
np.random.seed(42)

In [38]:
X_hot_model = np.load(join(FEATURES["RF_HoT"], "X_hot_NN.npy"))
Y_hot_model = np.load(join(FEATURES["RF_HoT"], "Y_hot_NN.npy"))

In [40]:
Y_hot_binary = np.zeros((Y_hot_model.shape[0],18))
i = 0
for y in Y_hot_model:
    Y_hot_binary[i][y-1] = 1
    i += 1

In [41]:
X_hot_model_train, X_hot_model_test, Y_hot_train, Y_hot_test = train_test_split(X_hot_model,Y_hot_binary)

In [81]:
model_HoT = Sequential()
model_HoT.add(Dense(32, input_shape=(108,), activation="relu", name='dense1'))
model_HoT.add(Dense(18, activation='softmax', name='dense_out'))
model_HoT.compile(optimizer='adam', loss="mse", metrics = ['accuracy'])
model_HoT.fit(X_hot_model, Y_hot_binary, epochs=100)

Epoch 1/100
18000/18000 [==============================] - 0s - loss: 0.0516 - acc: 0.1326         
Epoch 2/100
18000/18000 [==============================] - 0s - loss: 0.0500 - acc: 0.1800     

In [82]:
model_HoT.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 32)                3488      
_________________________________________________________________
dense_out (Dense)            (None, 18)                594       
Total params: 4,082.0
Trainable params: 4,082
Non-trainable params: 0.0
_________________________________________________________________


In [83]:
model_HoT.save_weights(join(FEATURES["RF_HoT"], "hot_NN_weights.h5"))

In [84]:
X = np.load(join(FEATURES["RF_HoT"], "X_RF_hot_NN.npy"))
Y = np.load(join(FEATURES["RF_HoT"], "Y_RF_hot_NN.npy"))

In [85]:
Y_binary = np.zeros((Y.shape[0],18))
i = 0
for y in Y:
    Y_binary[i][y-1] = 1
    i += 1

In [86]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y_binary) 

In [87]:
X_RF_train = X_train[:, :2700]
X_hot_train = X_train[:, 2700:]
X_RF_test = X_test[:, :2700]
X_hot_test = X_test[:, 2700:]

In [88]:
X_RF_train.shape, X_hot_train.shape, X_RF_test.shape,  X_hot_test.shape

((6714, 2700), (6714, 108), (2239, 2700), (2239, 108))

In [94]:
model_HoT_inner = Sequential()
model_HoT_inner.add(Dense(32, input_shape=(108,), activation="relu", name='dense1', trainable=False))
model_HoT_inner.load_weights(join(FEATURES["RF_HoT"], "hot_NN_weights.h5"), by_name=True)

In [95]:
model_RF = Sequential()
model_RF.add(Dense(32, input_shape=(2700,), activation="relu"))

model = Sequential()
model.add(Merge([model_RF, model_HoT_inner], mode='concat'))
model.add(Dense(18, activation='softmax'))
model.compile(optimizer='adam', loss="mse", metrics = ['accuracy'])
print(model.summary())
model.fit([X_RF_train, X_hot_train], Y_train, validation_data=([X_RF_test, X_hot_test], Y_test), epochs=10)

D:\Programs\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_8 (Merge)              (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 18)                1170      
Total params: 91,090.0
Trainable params: 87,602.0
Non-trainable params: 3,488.0
_________________________________________________________________
None
Train on 6714 samples, validate on 2239 samples
Epoch 1/10
6714/6714 [==============================] - 0s - loss: 0.0498 - acc: 0.1862 - val_loss: 0.0469 - val_acc: 0.2778

In [97]:
pred_binary = model.predict([X_RF_test, X_hot_test])